In [1]:
import requests
import json
import pandas as pd
import re
import robin_stocks
from robin_stocks import *
import robin_stocks.robinhood as r
import configur
import time

In [10]:
# Initiliazations for message list and stock dataframe
stocks_df = pd.DataFrame({'username': ['string username'],'message':['string message'],'timestamp':['string timestamp']})
message_list = {*()}

# Logs into Robinhood for portfolio managing
# print(r.login(username='', password='', expiresIn = 86400, by_sms= True,store_session=False))

# These are all from the discord rooms that we want messages from
message_room_ID_list = ['1226712190548180992','1135983105665159178','1130040044388884610','1128313244797382696',
                        '1154858987460775946','1090776258071240745','1219035439625928744','1217864641527550073',
                        '1133173656109993984','682259216861626378','1120826825313112114']

In [11]:
def robin_stocks_buy(ticker):
    open_positions = r.account.build_holdings()
    # print(open_positions)
    if ticker not in open_positions.keys():
        dictro = r.stocks.get_latest_price(ticker, priceType=None, includeExtendedHours=True)
        close = float(dictro[0])
        print (r.orders.order_buy_market(ticker, float("{:.1f}".format(40/close)),timeInForce='gfd'))
        time.sleep(12)  # Sleeps
        open_positions = r.account.build_holdings()
        if ticker not in open_positions.keys():
            r.cancel_all_stock_orders()
            robin_stocks_buy(ticker)

In [12]:
def robin_stocks_sell(ticker):
    open_positions = r.account.build_holdings()
    # print(open_positions)
    if ticker in open_positions.keys():
        dictro = r.stocks.get_latest_price(ticker, priceType=None, includeExtendedHours=True)
        close = float(dictro[0])
        print (r.orders.order_sell_market(ticker, float("{:.1f}".format(40/close)),timeInForce='gfd'))
        time.sleep(10)  # Sleeps
        open_positions = r.account.build_holdings()
        if ticker in open_positions.keys():
            r.cancel_all_stock_orders()
            robin_stocks_sell(ticker)

In [13]:
# Uses Requests package to get the last 50 messages of a Discord channel
def retrieve_messages(channel_id, last_message_id=None):
    global message_list
    content = ''
    auth_str = str(configur.thepass['disc_api'])
    headers = {'authorization': auth_str}
    url = f'https://discord.com/api/v9/channels/{channel_id}/messages'
    params = {
        'limit': 50  # You can adjust this number up to 100
    }
    if last_message_id:
        params['before'] = last_message_id  # Fetch messages before this ID

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        messages = response.json()
        for message in messages:
#             print(message)
            if len(message['content'].strip().split(' ')) > 1:
                content = message['content']
            else:
                try:
                    content = message['embeds'][0]['title']
                except Exception as e:
                    pass
            if content not in message_list and len(content) < 70:
                message_list.add(message['author']['username'] +' || '+ content.replace('|','')+' || '+str(message['timestamp']))

        # If you received messages, you can recursively call or iteratively loop to get more
        if messages:
            last_id = messages[-1]['id']  # ID of the last message in this batch
            retrieve_messages(channel_id, last_message_id=last_id)

In [14]:
def clean_message(message):
    # Centralized message cleaning
    for char in ['!', '.', ':', ',', '\r', '\n', '**']:
        message = message.replace(char, '')
    pattern = r"<@&\d+>"
    message = re.sub(pattern, "", message)
    return message.strip()

In [15]:
def update():
    global stocks_df
    global message_list
    # Uses the function defined above to get messages from multiple channels
    for msg in message_room_ID_list:
        try:
            retrieve_messages(msg)
        except Exception as e:
            print(e)

    for i in message_list:
        username, text, timestamp = i.split('||')[0], i.split('||')[1], i.split('||')[2]
        stock = ''
        iterator = len(stocks_df)
        text = clean_message(text)
        
        message = text.split(' ')
        if (len(message) <= 20):
            stocks_df.at[iterator,'username'] = username
            stocks_df.at[iterator, 'message'] = text
            stocks_df.at[iterator, 'timestamp'] = timestamp

    display(stocks_df)
    # Pushes df to excel
    stocks_df = stocks_df.iloc[1:, :]
    stocks_df.to_csv(r"C:\Users\amoog\Desktop\Project_X\Project_X\stocks_df.csv") 
         
update()          

,username,message,timestamp
0,string username,string message,string timestamp
1,Optionsking Alert Bot,Update,2023-10-30T14:11:21.612000+00:00
2,waysotheraccount,@everyone mdbh new ipo worth a try,2023-09-25T16:52:51.962000+00:00
3,Sir Goldman Alerts,COMMENT,2024-03-19T16:23:32.115000+00:00
4,𝗗𝗗 𝗔𝗟𝗘𝗥𝗧𝗦 🤴,__ ⚡️ NEW SIGNAL ⚡️__,2023-08-04T13:30:59.958000+00:00
...,...,...,...
8037,𝗗𝗗 𝗔𝗟𝗘𝗥𝗧𝗦 🤴,__ ⚡️ NEW SIGNAL ⚡️__,2023-09-27T17:12:16.442000+00:00
8038,Sir Goldman Alerts,COMMENT,2023-12-21T20:42:09.764000+00:00
8039,Bullish B Signals,ALL OUT SPY CALLS + 88% BOOOOOMMMM,2024-01-18T14:31:52.102000+00:00
8040,Waxui Alerts 🍭,Trim SPY here080 - 100 ✅ 25%Holding most,2024-03-28T13:41:36.081000+00:00


In [16]:
stocks_df.size

24123